# **<center>Random Experiment🎲<center>**

\** that makes no sense to me

This weekend I had nothing much to do and did not knew what to do. So I decided to do some random experiment with weights of simple neural network. I bacsicly trained two NN to classify 3 classes : 0,1 and 2. First model was trained on data containing 0 and 1 class only with just one class of 2.Second model was trained on data containing 0 and 2 class only with just one class of 1. Then I tried to combine the weights to improve the performce of the model .

## Imports

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
keras.utils.set_random_seed(69)

## Data Preparation

In [ ]:
data=pd.read_csv('/kaggle/input/random-exp-data/Random_Exp_Data.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
data.head()

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(data.loc[:,['C_0','C_1','C_2','C_3']], data.loc[:,'target'], test_size= 0.25, random_state=69)

In [ ]:
x_train_0and1=x_train[y_train != 2]
y_train_0and1=y_train[y_train != 2]
x_train_0and2=x_train[y_train != 1]
y_train_0and2=y_train[y_train != 1]

In [ ]:
x_train_0and1.loc[len(x_train_0and1)]=x_train_0and2.iloc[2]
y_train_0and1.loc[len(y_train_0and1)]=y_train_0and2.iloc[2]
x_train_0and2.loc[len(x_train_0and2)]=x_train_0and1.iloc[-1]
y_train_0and2.loc[len(y_train_0and2)]=y_train_0and1.iloc[-1]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train_0and1.to_numpy().reshape(-1, 1))
y_train_0and1=enc.transform(y_train_0and1.to_numpy().reshape(-1, 1))
y_train_0and2=enc.transform(y_train_0and2.to_numpy().reshape(-1, 1))
y_test=enc.transform(y_test.to_numpy().reshape(-1, 1))

## Model on 0 and 1 class

In [ ]:
model_0and1 = Sequential()
model_0and1.add(Dense(3, input_dim = 4, activation = 'relu'))
model_0and1.add(Dense(3, activation = 'relu'))
model_0and1.add(Dense(3, activation = 'softmax'))

In [ ]:
model_0and1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_0and1.fit(x_train_0and1, y_train_0and1.toarray(), epochs = 15, batch_size = 5, verbose = 0)
scores = model_0and1.evaluate(x_test, y_test.toarray())

In [ ]:
weights_0and1 = np.array(model_0and1.get_weights(),dtype=object)
weights_0and1

## Model on 0 and 2 class

In [ ]:
model_0and2 = Sequential()
model_0and2.add(Dense(3, input_dim = 4, activation = 'relu'))
model_0and2.add(Dense(3, activation = 'relu'))
model_0and2.add(Dense(3, activation = 'softmax'))

In [ ]:
model_0and2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_0and2.fit(x_train_0and2, y_train_0and2.toarray(), epochs = 15, batch_size = 5, verbose=0)
scores = model_0and1.evaluate(x_test, y_test.toarray())

In [ ]:
weights_0and2 = np.array(model_0and2.get_weights(),dtype=object)
weights_0and2

## Combined model :

In [ ]:
model_combined = Sequential()
model_combined.add(Dense(3, input_dim = 4, activation = 'relu'))
model_combined.add(Dense(3, activation = 'relu'))
model_combined.add(Dense(3, activation = 'softmax'))

model_combined.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

## Combining Weights :

- ### $w = \alpha	*w_{0and1} + (1-\alpha	)*w_{0and2}$

In [ ]:
alpha_list=[]
accuracy_list=[]
for i in range(0,11,1):
    alpha=i/10
    weight_combined=[]
    for i in range(len(weights_0and1)):
        weight_combined.append(alpha*weights_0and1[i]+(1-alpha)*weights_0and2[i])
    weight_combined=np.array(weight_combined,dtype=object)
    model_combined.set_weights(weight_combined)
    scores = model_combined.evaluate(x_test, y_test.toarray(), verbose=0)
    alpha_list.append(alpha)
    accuracy_list.append(scores[1])
plt.plot(alpha_list,accuracy_list)

Clearly combining model in such a way was not a good idea as the accuracy droped significantly when alpha=0.5 .

- ### $w = \alpha	*w^2_{0and1} + (1-\alpha)*w^2_{0and2}$

In [ ]:
alpha_list=[]
accuracy_list=[]
for i in range(0,11,1):
    alpha=i/10
    weight_combined=[]
    for i in range(len(weights_0and1)):
        weight_combined.append(alpha*weights_0and1[i]**2+(1-alpha)*weights_0and2[i]**2)
    weight_combined=np.array(weight_combined,dtype=object)
    model_combined.set_weights(weight_combined)
    scores = model_combined.evaluate(x_test, y_test.toarray(),verbose=0)
    alpha_list.append(alpha)
    accuracy_list.append(scores[1])
plt.plot(alpha_list,accuracy_list)

Again I got terrible results 🥲.

- ### $w = \frac{w^n_{0and1} +w^n_{0and2}}{2}$

In [ ]:
accuracy_list=[]
for n in range(0,100,1):
    weight_combined=[]
    for i in range(len(weights_0and1)):
        weight_combined.append((weights_0and1[i]**n+weights_0and2[i]**n)/2)
    weight_combined=np.array(weight_combined,dtype=object)
    model_combined.set_weights(weight_combined)
    scores = model_combined.evaluate(x_test, y_test.toarray(),verbose=0)
    accuracy_list.append(scores[1])
plt.plot(accuracy_list)

In [ ]:
plt.plot(accuracy_list[:10])

Now I got something intresting . These spikes are very much intresting and till now I have not find any reason to explain this . The pattern is that there is a spike in accuracy at **even values of n** and have **higher accuracies** as compared to nearby *odd values of n*.

### I would love to know what is the reason behind this pattern or to know that this is a just a random pattern which I have found.